In [1]:
import pandas as pd

In [2]:
# newest = pd.read_csv("fastapi-server/newest_data.csv")
forecasted_data = pd.read_csv("fastapi-server/forecasted_pollutant.csv")
aqi_7_days_lag_df = pd.read_csv("fastapi-server/aqi_7_days_lag.csv")
aqi_14_days_lag_df = pd.read_csv("fastapi-server/aqi_14_days_lag.csv")

In [3]:
forecasted_data.head()


,Unnamed: 0,Aqi,Carbon_monoxide,District,Dust,Location_id,Nitrogen_dioxide,Ozone,Pm_10,Pm_25,Sulphur_dioxide
0,2024-09-25 00:00:00,85.30002,520.99270,Khushab,1.101463,1.0,9.541286,78.964485,49.821790,41.786720,9.159114
1,2024-09-25 01:00:00,82.62073,501.01584,Khushab,0.731753,1.0,6.442183,82.532196,47.407780,41.976543,9.169228
2,2024-09-25 02:00:00,80.33057,482.30542,Khushab,0.537461,1.0,4.075489,90.485245,44.037773,41.864227,9.378104
3,2024-09-25 03:00:00,78.74734,469.24216,Khushab,0.385919,1.0,1.970705,98.765610,41.015290,41.555650,9.889211
4,2024-09-25 04:00:00,77.34331,456.86554,Khushab,0.280189,1.0,1.124473,105.741600,38.064625,41.421650,10.283500


In [4]:
# newest.tail()

In [6]:
forecasted_data.head()
# renaming the unnamed: 0 column to date
forecasted_data.rename(columns = {'Unnamed: 0':'date'}, inplace = True)
forecasted_data.head()



,date,Aqi,Carbon_monoxide,District,Dust,Location_id,Nitrogen_dioxide,Ozone,Pm_10,Pm_25,Sulphur_dioxide
0,2024-09-25 00:00:00,85.30002,520.99270,Khushab,1.101463,1.0,9.541286,78.964485,49.821790,41.786720,9.159114
1,2024-09-25 01:00:00,82.62073,501.01584,Khushab,0.731753,1.0,6.442183,82.532196,47.407780,41.976543,9.169228
2,2024-09-25 02:00:00,80.33057,482.30542,Khushab,0.537461,1.0,4.075489,90.485245,44.037773,41.864227,9.378104
3,2024-09-25 03:00:00,78.74734,469.24216,Khushab,0.385919,1.0,1.970705,98.765610,41.015290,41.555650,9.889211
4,2024-09-25 04:00:00,77.34331,456.86554,Khushab,0.280189,1.0,1.124473,105.741600,38.064625,41.421650,10.283500


In [7]:
# filtering both data to have only the date, District and Aqi columns
# newest_filtered = newest[['date', 'District', 'Aqi']]
forecasted_data_filtered = forecasted_data[['date', 'District', 'Aqi']]

# concatenating the filtered dataframes
combined_data = forecasted_data_filtered
combined_data.head()


,date,District,Aqi
0,2024-09-25 00:00:00,Khushab,85.30002
1,2024-09-25 01:00:00,Khushab,82.62073
2,2024-09-25 02:00:00,Khushab,80.33057
3,2024-09-25 03:00:00,Khushab,78.74734
4,2024-09-25 04:00:00,Khushab,77.34331


In [8]:
combined_data

,date,District,Aqi
0,2024-09-25 00:00:00,Khushab,85.300020
1,2024-09-25 01:00:00,Khushab,82.620730
2,2024-09-25 02:00:00,Khushab,80.330570
3,2024-09-25 03:00:00,Khushab,78.747340
4,2024-09-25 04:00:00,Khushab,77.343310
...,...,...,...
647995,2024-12-23 19:00:00,Mianwali,33.561737
647996,2024-12-23 20:00:00,Mianwali,33.561737
647997,2024-12-23 21:00:00,Mianwali,33.561740
647998,2024-12-23 22:00:00,Mianwali,33.561745


In [9]:
# converting the date column to datetime
combined_data['date'] = pd.to_datetime(combined_data['date'])

combined_data.head()

C:\Users\timmy_3aupohg\AppData\Local\Temp\ipykernel_32040\1197255911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data['date'] = pd.to_datetime(combined_data['date'])


,date,District,Aqi
0,2024-09-25 00:00:00,Khushab,85.30002
1,2024-09-25 01:00:00,Khushab,82.62073
2,2024-09-25 02:00:00,Khushab,80.33057
3,2024-09-25 03:00:00,Khushab,78.74734
4,2024-09-25 04:00:00,Khushab,77.34331


In [10]:
# converting the data to daily by selecting the max in the day for that day
# this should be done for each district
# Group by 'district' and 'date', then select the max value for each group
daily_max = combined_data.groupby(['District', combined_data['date'].dt.date])['Aqi'].max().reset_index()

# Rename columns for clarity
daily_max.columns = ['District', 'date', 'Aqi']

daily_max.head()


,District,date,Aqi
0,Attock,2024-09-25,139.607510
1,Attock,2024-09-26,105.255040
2,Attock,2024-09-27,68.881660
3,Attock,2024-09-28,61.563870
4,Attock,2024-09-29,56.632793


In [11]:
daily_max.shape

(3240, 3)

In [12]:
# saving the daily_max dataframe to a csv file
daily_max.to_csv("daily_historical_data.csv", index=False)

In [13]:
# creating a daily max version for the forecasted data
# selecting the columns to work with
forecasted_data_filtered = forecasted_data[['date', 'District', 'Aqi']]

# converting the date column to datetime
forecasted_data_filtered['date'] = pd.to_datetime(forecasted_data_filtered['date'])

# converting the data to daily by selecting the max in the day for that day
# this should be done for each district
daily_max_forecasted = forecasted_data_filtered.groupby(['District', forecasted_data_filtered['date'].dt.date])['Aqi'].max().reset_index()

# Rename columns for clarity
daily_max_forecasted.columns = ['District', 'date', 'Aqi']

daily_max_forecasted.head()

C:\Users\timmy_3aupohg\AppData\Local\Temp\ipykernel_32040\2306417717.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecasted_data_filtered['date'] = pd.to_datetime(forecasted_data_filtered['date'])


,District,date,Aqi
0,Attock,2024-09-25,139.607510
1,Attock,2024-09-26,105.255040
2,Attock,2024-09-27,68.881660
3,Attock,2024-09-28,61.563870
4,Attock,2024-09-29,56.632793


In [14]:
# checking the shape of the daily_max_forecasted dataframe
daily_max_forecasted.shape

(3240, 3)

In [15]:
# saving the daily_max_forecasted dataframe to a csv file
daily_max_forecasted.to_csv("daily_forecasted_data.csv", index=False)

In [16]:
# creating a daily max version for the 7 days lag data
# selecting the columns to work with
aqi_7_days_lag_df = aqi_7_days_lag_df[['date', 'district', 'Aqi']]

# converting the date column to datetime
aqi_7_days_lag_df['date'] = pd.to_datetime(aqi_7_days_lag_df['date'])

# converting the data to daily by selecting the max in the day for that day 
daily_max_7_days_lag = aqi_7_days_lag_df.groupby(['district', aqi_7_days_lag_df['date'].dt.date])['Aqi'].max().reset_index()

# Rename columns for clarity
daily_max_7_days_lag.columns = ['district', 'date', 'Aqi']

daily_max_7_days_lag.head()
# saving the daily_max_7_days_lag dataframe to a csv file
daily_max_7_days_lag.to_csv("daily_7_days_lag_data.csv", index=False)
daily_max_7_days_lag = aqi_7_days_lag_df.groupby(['district', aqi_7_days_lag_df['date'].dt.date])['Aqi'].max().reset_index()



In [17]:
daily_max_7_days_lag

,district,date,Aqi
0,Attock,2024-09-18,102.954770
1,Attock,2024-09-19,84.964005
2,Attock,2024-09-20,87.329796
3,Attock,2024-09-21,83.678734
4,Attock,2024-09-22,81.499320
...,...,...,...
3487,Vehari,2024-12-19,147.341580
3488,Vehari,2024-12-20,147.341670
3489,Vehari,2024-12-21,147.341700
3490,Vehari,2024-12-22,147.341630


In [18]:
# doing the same for the 14 days lag data
# selecting the columns to work with
aqi_14_days_lag_df = aqi_14_days_lag_df[['date', 'district', 'Aqi']]

# converting the date column to datetime
aqi_14_days_lag_df['date'] = pd.to_datetime(aqi_14_days_lag_df['date'])

# converting the data to daily by selecting the max in the day for that day 
daily_max_14_days_lag = aqi_14_days_lag_df.groupby(['district', aqi_14_days_lag_df['date'].dt.date])['Aqi'].max().reset_index() 

# Rename columns for clarity
daily_max_14_days_lag.columns = ['district', 'date', 'Aqi']

daily_max_14_days_lag.head()
# saving the daily_max_14_days_lag dataframe to a csv file
daily_max_14_days_lag.to_csv("daily_14_days_lag_data.csv", index=False)


In [19]:
daily_max_14_days_lag

,district,date,Aqi
0,Attock,2024-09-11,112.166810
1,Attock,2024-09-12,101.989460
2,Attock,2024-09-13,101.244120
3,Attock,2024-09-14,100.528340
4,Attock,2024-09-15,100.160866
...,...,...,...
3739,Vehari,2024-12-19,110.191414
3740,Vehari,2024-12-20,110.191400
3741,Vehari,2024-12-21,110.191420
3742,Vehari,2024-12-22,110.191420
